In [1]:
import os
cache_dir = '/scratch3/workspace/wenlongzhao_umass_edu-reason/dev_kedar/transformers_cache'
os.environ['TRANSFORMERS_CACHE'] = cache_dir
os.environ['HF_HOME']=cache_dir
os.environ['HF_HUB_CACHE']=cache_dir+'/hub'

In [2]:
import argparse
import torch
import torch.nn.functional as F
from tqdm import tqdm
import numpy as np
from datasets import load_from_disk, load_dataset
# from small_llm_reasoning.generation.transformers_generation import generation
from transformers import AutoModelForCausalLM, AutoTokenizer


/work/pi_mccallum_umass_edu/kchimmad_umass_edu/conda_envs/reason/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/work/pi_mccallum_umass_edu/kchimmad_umass_edu/conda_envs/reason/lib/python3.12/site-packages/transformers/utils/hub.py:128: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [4]:
student_data = load_dataset('json',data_files='/scratch3/workspace/wenlongzhao_umass_edu-reason/dev_kedar/Small-LLM-Reasoning/outputs/exp-2.0.1/eval_1/generated_outputs.json')['train']
student_data

Dataset({
    features: ['input', 'output', 'token_ids', 'log_probs', 'all_returned_log_probs', 'model_answer', 'GT_Answer', 'score'],
    num_rows: 1000
})

In [5]:
# Loading data
data_path= "../datasets/gsm8k"
split='feedback'
data = load_from_disk(f"{data_path}/tokenized/LLaMA3B-Instruct/{split}/zero-shot/")
data

Dataset({
    features: ['question', 'answer', 'input_ids'],
    num_rows: 1000
})

In [6]:
model_name= "meta-llama/Llama-3.2-3B-Instruct"
padding='longest'
padding_side=None
special_tokens=False
torch_dtype='bfloat16'
hf_token=os.getenv('hf_token')

In [7]:
tokenizer = AutoTokenizer.from_pretrained(
        model_name,
        token=hf_token, 
        cache_dir=cache_dir
    )
tokenizer.pad_token_id = tokenizer.eos_token_id

model = AutoModelForCausalLM.from_pretrained(
    model_name, 
    device_map="auto", 
    torch_dtype=torch_dtype,
    token=hf_token, 
    cache_dir=cache_dir
)

if padding_side:
        tokenizer.padding_side = padding_side
add_special_tokens = {"add_special_tokens": special_tokens}

Could not cache non-existence of file. Will ignore error and continue. Error: [Errno 13] Permission denied: '/scratch3/workspace/wenlongzhao_umass_edu-reason/dev_kedar/transformers_cache/models--meta-llama--Llama-3.2-3B-Instruct/.no_exist/0cb88a4f764b7a12671c53f0838cd831a0843b95/adapter_config.json'
Could not cache non-existence of file. Will ignore error and continue. Error: [Errno 13] Permission denied: '/scratch3/workspace/wenlongzhao_umass_edu-reason/dev_kedar/transformers_cache/models--meta-llama--Llama-3.2-3B-Instruct/.no_exist/0cb88a4f764b7a12671c53f0838cd831a0843b95/adapter_config.json'
Loading checkpoint shards: 100%|██████████| 2/2 [00:38<00:00, 19.45s/it]


In [8]:
def combine_data(q,a):
    return q+a
    

In [24]:
tokenizer('abcde', padding=padding, return_tensors="pt", **add_special_tokens).to(model.device)


{'input_ids': tensor([[13997,   451]], device='cuda:0'), 'attention_mask': tensor([[1, 1]], device='cuda:0')}

In [46]:
answer_log_probs = []
for i in range(data.num_rows):
    question= data['input_ids'][i]['prompt_token_ids']
    answer= student_data['token_ids'][i][0] 
    example= question + answer
    example=torch.tensor(example, dtype=torch.long).unsqueeze(0).to(model.device)
    answer=[answer]
    # Forward pass
    with torch.no_grad():
        outputs = model(example)
    
    # Extract logits
    logits = outputs.logits  # Shape: [batch_size, seq_len, vocab_size]
    
    
    # Convert logits to log probabilities
    log_probs = F.log_softmax(logits,dim=-1) # Shape: [batch_size, seq_len, vocab_size]
    
    # Identify the token IDs for the answer (without special tokens)
    
    answer_tokens = []
    answer_token_idxs=[]
    
    
    for idx, (prompt, answer_tokens_ids) in enumerate(zip(example, answer)):
        # Extract token IDs for answer tokens in the concatenated input
        print(f'IDX:\n{idx}')
        print(f'Prompts:\n{prompt}')
        print(f'Answer_Token_Ids:\n{answer_tokens_ids}')
        answer_start_idx = (example[idx] == answer_tokens_ids[0]).nonzero(as_tuple=True)[0][0].item()
        answer_end_idx = answer_start_idx + len(answer_tokens_ids)
        answer_tokens.append(example[idx, answer_start_idx:answer_end_idx])
        print(f'Answer:\n{tokenizer.decode(example[idx, answer_start_idx:answer_end_idx], skip_special_tokens=True)}')
        # answer_token_idxs.append(list(range(answer_start_idx, answer_end_idx)))
        answer_token_idxs.append([answer_start_idx,answer_end_idx])

    # Now, extract the log probs for the selected tokens (answer tokens)
    for i, tokens in enumerate(answer_tokens):
        # answer_positions = torch.tensor(answer_token_idxs[i], device=model.device)  # Convert to tensor
        selected_log_probs = log_probs[i, answer_token_idxs[i][0]:answer_token_idxs[i][1]].gather(dim=-1, index=tokens.unsqueeze(-1)).squeeze(-1)
        answer_log_probs.append(selected_log_probs.tolist())
    print(answer_log_probs)
    print(len(answer_log_probs[i]))
    print(len(student_data['log_probs'][i][0]))
    break

IDX:
0
Prompts:
tensor([128000, 128006,   9125, 128007,    271,  38766,   1303,  33025,   2696,
            25,   6790,    220,   2366,     18,    198,  15724,   2696,     25,
           220,    605,   2947,    220,   2366,     20,    271, 128009, 128006,
           882, 128007,    271,  22818,    279,   2768,   3575,     11,   2944,
           323,   3041,    264,   1620,   4320,    311,    279,   3575,    627,
         32298,     25,    362,   3830,    374,    220,     23,  15271,    304,
          2673,     11,    220,    605,  15271,    304,   2430,     11,    323,
           220,    717,  15271,    304,   3160,     13,    362,  23162,   4857,
          2565,    374,    220,     18,  15271,    304,   2673,     11,    220,
            17,  15271,    304,   2430,     11,    323,    220,     19,  15271,
           304,   3160,     13,   2650,   1690,   4857,  10215,    649,   5052,
          1139,    279,   3830,   5380,   7927,   2077,   1288,    842,    449,
           330,    791, 

In [12]:
for i in range(data.num_rows):
    assert len(data[i]['log_probs'][0])==len(data[i]['all_returned_log_probs'])
    assert len(data[i]['log_probs'][0])==len(data[i]['token_ids'][0])

In [50]:
model_name= "meta-llama/Llama-3.2-3B-Instruct"
padding='longest'
padding_side=None
special_tokens=False
torch_dtype='bfloat16'
hf_token=os.getenv('hf_token')

In [51]:
tokenizer = AutoTokenizer.from_pretrained(
        model_name,
        token=hf_token, 
        cache_dir=cache_dir
    )
tokenizer.pad_token_id = tokenizer.eos_token_id

model = AutoModelForCausalLM.from_pretrained(
    model_name, 
    device_map="auto", 
    torch_dtype=torch_dtype,
    token=hf_token, 
    cache_dir=cache_dir
)

if padding_side:
        tokenizer.padding_side = padding_side
add_special_tokens = {"add_special_tokens": special_tokens}

Could not cache non-existence of file. Will ignore error and continue. Error: [Errno 13] Permission denied: '/scratch3/workspace/wenlongzhao_umass_edu-reason/dev_kedar/transformers_cache/models--meta-llama--Llama-3.2-3B-Instruct/.no_exist/0cb88a4f764b7a12671c53f0838cd831a0843b95/adapter_config.json'
Could not cache non-existence of file. Will ignore error and continue. Error: [Errno 13] Permission denied: '/scratch3/workspace/wenlongzhao_umass_edu-reason/dev_kedar/transformers_cache/models--meta-llama--Llama-3.2-3B-Instruct/.no_exist/0cb88a4f764b7a12671c53f0838cd831a0843b95/adapter_config.json'
Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.32it/s]


In [52]:
def combine_data(q,a):
    return q+a
    

In [53]:
# HF
output=tokenizer(data['output'][0][0],return_tensors="pt", **add_special_tokens)
tokens=output['input_ids'].tolist()[0]
print(tokens)
print(len(tokens))
print(tokenizer.decode(tokens, skip_special_tokens=False))

[1271, 1505, 704, 1268, 1690, 4857, 10215, 649, 5052, 1139, 279, 3830, 11, 584, 1205, 311, 22497, 279, 8286, 315, 279, 3830, 555, 279, 8286, 315, 264, 3254, 4857, 2565, 382, 791, 8286, 315, 279, 3830, 374, 16997, 555, 85292, 1202, 2673, 11, 2430, 11, 323, 3160, 512, 19436, 315, 3830, 284, 2673, 353, 2430, 353, 3160, 284, 220, 23, 353, 220, 605, 353, 220, 717, 284, 220, 16415, 41999, 15271, 271, 791, 8286, 315, 264, 3254, 4857, 2565, 374, 16997, 555, 85292, 1202, 2673, 11, 2430, 11, 323, 3160, 512, 19436, 315, 4857, 2565, 284, 2673, 353, 2430, 353, 3160, 284, 220, 18, 353, 220, 17, 353, 220, 19, 284, 220, 1187, 41999, 15271, 271, 7184, 11, 584, 22497, 279, 8286, 315, 279, 3830, 555, 279, 8286, 315, 264, 3254, 4857, 2565, 311, 1505, 704, 1268, 1690, 10215, 649, 5052, 512, 2903, 315, 10215, 284, 20880, 315, 3830, 611, 20880, 315, 4857, 2565, 284, 220, 16415, 611, 220, 1187, 284, 220, 1272, 271, 791, 1620, 4320, 374, 220, 1272, 13]
171
To find out how many building blocks can fit into the 

In [54]:
# vLLM
print(data['token_ids'][0][0])
print(len(data['token_ids'][0][0]))
print(tokenizer.decode(data['token_ids'][0][0], skip_special_tokens=False))

[1271, 1505, 704, 1268, 1690, 4857, 10215, 649, 5052, 1139, 279, 3830, 11, 584, 1205, 311, 22497, 279, 8286, 315, 279, 3830, 555, 279, 8286, 315, 264, 3254, 4857, 2565, 382, 791, 8286, 315, 279, 3830, 374, 16997, 555, 85292, 1202, 2673, 11, 2430, 11, 323, 3160, 512, 19436, 315, 3830, 284, 2673, 353, 2430, 353, 3160, 284, 220, 23, 353, 220, 605, 353, 220, 717, 284, 220, 16415, 41999, 15271, 271, 791, 8286, 315, 264, 3254, 4857, 2565, 374, 16997, 555, 85292, 1202, 2673, 11, 2430, 11, 323, 3160, 512, 19436, 315, 4857, 2565, 284, 2673, 353, 2430, 353, 3160, 284, 220, 18, 353, 220, 17, 353, 220, 19, 284, 220, 1187, 41999, 15271, 271, 7184, 11, 584, 22497, 279, 8286, 315, 279, 3830, 555, 279, 8286, 315, 264, 3254, 4857, 2565, 311, 1505, 704, 1268, 1690, 10215, 649, 5052, 512, 2903, 315, 10215, 284, 20880, 315, 3830, 611, 20880, 315, 4857, 2565, 284, 220, 16415, 611, 220, 1187, 284, 220, 1272, 271, 791, 1620, 4320, 374, 220, 1272, 13, 128009]
172
To find out how many building blocks can fit i

In [55]:
print(data['output'][0][0])

To find out how many building blocks can fit into the box, we need to divide the volume of the box by the volume of a single building block.

The volume of the box is calculated by multiplying its height, width, and length:
Volume of box = height * width * length = 8 * 10 * 12 = 960 cubic inches

The volume of a single building block is calculated by multiplying its height, width, and length:
Volume of building block = height * width * length = 3 * 2 * 4 = 24 cubic inches

Now, we divide the volume of the box by the volume of a single building block to find out how many blocks can fit:
Number of blocks = Volume of box / Volume of building block = 960 / 24 = 40

The final answer is 40.


In [56]:
# HF
output=tokenizer(data['output'][143][0],return_tensors="pt", **add_special_tokens)
tokens=output['input_ids'].tolist()[0]
print(tokens)
print(len(tokens))
print(tokenizer.decode(tokens, skip_special_tokens=False))

[1271, 1505, 704, 1268, 1690, 2678, 6136, 388, 15796, 41033, 3966, 11, 584, 1176, 1205, 311, 8417, 1268, 1690, 19595, 568, 649, 6136, 304, 279, 3544, 6136, 388, 13, 4815, 1548, 706, 220, 19, 3544, 6136, 388, 430, 649, 3412, 220, 508, 19595, 1855, 11, 779, 279, 2860, 1396, 315, 19595, 568, 649, 6136, 304, 279, 3544, 6136, 388, 374, 512, 19, 6136, 388, 353, 220, 508, 19595, 14, 10609, 466, 284, 220, 1490, 19595, 271, 7184, 11, 584, 33356, 279, 19595, 39441, 304, 279, 3544, 6136, 388, 505, 279, 2860, 1396, 315, 19595, 512, 1049, 19595, 482, 220, 1490, 19595, 284, 220, 4364, 19595, 271, 12834, 279, 2678, 6136, 388, 649, 3412, 220, 19, 19595, 1855, 11, 584, 22497, 279, 9861, 19595, 555, 220, 19, 311, 1505, 704, 1268, 1690, 2678, 6136, 388, 527, 4460, 512, 4364, 19595, 611, 220, 19, 19595, 14, 10609, 466, 284, 220, 966, 6136, 388, 271, 791, 1620, 4320, 374, 220, 966, 13]
159
To find out how many small planters Oshea needs, we first need to determine how many seeds he can plant in the large p

In [64]:
tokenizer.decode([14 ,10609 ,466])

'/planter'

In [65]:
tokenizer.decode([11505 ,81130])

'/planter'

In [57]:
# vLLM
print(data['token_ids'][143][0])
print(len(data['token_ids'][143][0]))
print(tokenizer.decode(data['token_ids'][143][0], skip_special_tokens=False))

[1271, 1505, 704, 1268, 1690, 2678, 6136, 388, 15796, 41033, 3966, 11, 584, 1176, 1205, 311, 8417, 1268, 1690, 19595, 568, 649, 6136, 304, 279, 3544, 6136, 388, 13, 4815, 1548, 706, 220, 19, 3544, 6136, 388, 430, 649, 3412, 220, 508, 19595, 1855, 11, 779, 279, 2860, 1396, 315, 19595, 568, 649, 6136, 304, 279, 3544, 6136, 388, 374, 512, 19, 6136, 388, 353, 220, 508, 19595, 11505, 81130, 284, 220, 1490, 19595, 271, 7184, 11, 584, 33356, 279, 19595, 39441, 304, 279, 3544, 6136, 388, 505, 279, 2860, 1396, 315, 19595, 512, 1049, 19595, 482, 220, 1490, 19595, 284, 220, 4364, 19595, 271, 12834, 279, 2678, 6136, 388, 649, 3412, 220, 19, 19595, 1855, 11, 584, 22497, 279, 9861, 19595, 555, 220, 19, 311, 1505, 704, 1268, 1690, 2678, 6136, 388, 527, 4460, 512, 4364, 19595, 611, 220, 19, 19595, 11505, 81130, 284, 220, 966, 6136, 388, 271, 791, 1620, 4320, 374, 220, 966, 13, 128009]
158
To find out how many small planters Oshea needs, we first need to determine how many seeds he can plant in the lar

In [58]:
print(data['output'][0][0])

To find out how many building blocks can fit into the box, we need to divide the volume of the box by the volume of a single building block.

The volume of the box is calculated by multiplying its height, width, and length:
Volume of box = height * width * length = 8 * 10 * 12 = 960 cubic inches

The volume of a single building block is calculated by multiplying its height, width, and length:
Volume of building block = height * width * length = 3 * 2 * 4 = 24 cubic inches

Now, we divide the volume of the box by the volume of a single building block to find out how many blocks can fit:
Number of blocks = Volume of box / Volume of building block = 960 / 24 = 40

The final answer is 40.


In [16]:

batch_size=1
answer_log_probs = []
for i in tqdm(range(0,data.num_rows,batch_size)):
    questions=data['input'][i:i+batch_size]
    
    prompts=[]
    answers=[]
    for j in range(len(questions)):
        answers.append(data['output'][i+j][0])
        prompts.append(combine_data(questions[j],answers[j]))
    # print(answers)
    # print(f'Answers:\n{answers}')
    # Tokenize with padding for batch input
    inputs = tokenizer(prompts, padding=padding, return_tensors="pt", **add_special_tokens).to(model.device)

    # output=generation(prompts, model_name, padding, padding_side, torch_dtype, special_tokens)
   
    # Forward pass
    with torch.no_grad():
        outputs = model(**inputs)
        
    # Extract logits
    logits = outputs.logits  # Shape: [batch_size, seq_len, vocab_size]
    
    
    # Convert logits to log probabilities
    log_probs = F.log_softmax(logits,dim=-1) # Shape: [batch_size, seq_len, vocab_size]
    # print(f'log_probs:\n{log_probs}')
    # print(f'Shape of log_probs: {log_probs.shape}')

    
    # Identify the token IDs for the answer (without special tokens)
    answer_token_ids = [tokenizer(a, add_special_tokens=False)["input_ids"] for a in answers]
    answer_tokens = []
    answer_token_idxs=[]
    
    
    for idx, (prompts, answer_tokens_ids) in enumerate(zip(prompts, answer_token_ids)):
        # Extract token IDs for answer tokens in the concatenated input
        # print(f'IDX:\n{idx}')
        # print(f'Prompts:\n{prompts}')
        # print(f'Answer_Token_Ids:\n{answer_token_ids}')
        # print(f'Input Ids:\n{inputs["input_ids"][idx]}')
        answer_start_idx = (inputs["input_ids"][idx] == answer_tokens_ids[0]).nonzero(as_tuple=True)[0][0].item()
        answer_end_idx = answer_start_idx + len(answer_tokens_ids)
        answer_tokens.append(inputs["input_ids"][idx, answer_start_idx:answer_end_idx])
        # print(f'Answer:\n{tokenizer.decode(inputs["input_ids"][idx, answer_start_idx:answer_end_idx], skip_special_tokens=True)}')
        # answer_token_idxs.append(list(range(answer_start_idx, answer_end_idx)))
        answer_token_idxs.append([answer_start_idx,answer_end_idx])

    # Now, extract the log probs for the selected tokens (answer tokens)
    for i, tokens in enumerate(answer_tokens):
        # answer_positions = torch.tensor(answer_token_idxs[i], device=model.device)  # Convert to tensor
        selected_log_probs = log_probs[i, answer_token_idxs[i][0]:answer_token_idxs[i][1]].gather(dim=-1, index=tokens.unsqueeze(-1)).squeeze(-1)
        answer_log_probs.append(selected_log_probs.tolist())

    # # Now, extract the log probs for the selected tokens (answer tokens)
    # for i, tokens in enumerate(answer_tokens):
    #     answer_positions = torch.tensor(answer_token_idxs[i], device=model.device)  # Convert to tensor
    #     selected_log_probs = log_probs[i, answer_positions].gather(dim=-1, index=tokens.unsqueeze(-1)).squeeze(-1)
    #     answer_log_probs.append(selected_log_probs.tolist())
    
#     # Print log probabilities for the answer tokens
#     for idx, log_probs in enumerate(answer_log_probs):
#         print(f"Log probabilities for answer {idx}: {log_probs}")
    
    
    # break
    

100%|██████████| 63/63 [04:15<00:00,  4.05s/it]


In [16]:
len(answer_log_probs)

1000

In [17]:
data = data.add_column("log_probs_teacher", answer_log_probs)

In [18]:
data

Dataset({
    features: ['input', 'output', 'log_probs', 'model_answer', 'GT_Answer', 'score', 'log_probs_teacher'],
    num_rows: 1000
})

In [19]:
# import json
# with open("dataset.jsonl", "w") as f:
#     for row in data:
#         f.write(json.dumps(row) + "\n")

In [ ]:
data

In [23]:
for i in range(data.num_rows):
    print(len(data['log_probs'][i][0]))
    print(len(data['log_probs_teacher'][i]))
    print(i)
    assert len(data['log_probs'][i][0])-1==len(data['log_probs_teacher'][i])
    
    
    
    # break
    

172
171
0
98
97
1
194
193
2
255
254
3
126
125
4
154
153
5
340
339
6
83
82
7
174
173
8
158
157
9
245
244
10
156
155
11
196
195
12
153
152
13
207
206
14
186
185
15
199
198
16
167
166
17
150
149
18
187
186
19
149
148
20
177
176
21
126
125
22
142
141
23
165
164
24
176
175
25
168
167
26
116
115
27
284
283
28
223
222
29
94
93
30
356
355
31
225
224
32
154
153
33
129
128
34
315
314
35
127
126
36
166
165
37
211
210
38
182
181
39
143
142
40
295
294
41
141
140
42
154
153
43
239
238
44
90
89
45
131
130
46
242
241
47
160
159
48
225
224
49
326
325
50
156
155
51
196
195
52
116
115
53
196
195
54
170
169
55
171
170
56
123
122
57
225
224
58
194
193
59
135
134
60
285
284
61
256
255
62
172
171
63
202
201
64
106
105
65
214
213
66
188
187
67
140
139
68
139
138
69
90
89
70
199
198
71
136
135
72
223
222
73
138
137
74
114
113
75
204
203
76
171
170
77
197
196
78
99
98
79
165
164
80
124
123
81
206
205
82
233
232
83
213
212
84
204
203
85
130
129
86
178
177
87
130
129
88
99
98
89
130
129
90
362
361
91
256
255
92
2

AssertionError: 

In [25]:
print(data['log_probs'][i][0]) # vLLM

[-0.016434656456112, -0.352039039134979, -0.313303917646408, -0.029853513464331002, -3.93382906622719e-05, -0.20064103603363, -0.00027926836628400003, -4.6491513785440475e-06, -0.003412378486245, -3.182837463100441e-05, -0.633067727088928, -0.007620431482791, -0.39467477798461903, -0.207727447152137, -0.026530675590038, 0.0, -0.8062194585800171, -0.012836328707635, -0.00030286493711100004, -0.018957853317260003, -0.44235873222351, -0.23087894916534402, -0.06290937215089701, -0.044039089232683, -0.081897713243961, -0.0021127776708450004, -0.00037222131504600003, -3.576278118089249e-07, -0.513729095458984, -1.017504215240478, -1.035511136054992, -0.004137171432375001, -1.311293544858926e-05, -0.000178440386662, -7.033323527139146e-06, -2.7298554414301183e-05, -1.430510451427835e-06, -0.43764889240264804, -0.005467580165714, -0.016074467450380003, -5.006777428206988e-06, -9.179073458653875e-06, -2.145764938177308e-06, -7.152531907195225e-06, -0.576463043689727, -0.005884229205548001, -0.8

In [26]:
print(data['log_probs_teacher'][i]) # HF

[-26.625, -18.5, -16.25, -19.25, -15.0, -24.125, -25.25, -23.75, -29.375, -26.125, -22.5, -29.875, -16.0, -13.0, -24.5, -17.375, -20.375, -18.875, -16.75, -12.75, -14.8125, -15.9375, -21.125, -16.0, -18.125, -17.375, -27.875, -22.0, -31.875, -20.0, -24.125, -17.5, -19.625, -31.375, -20.75, -26.875, -23.75, -19.375, -17.75, -22.625, -20.0, -32.0, -23.625, -21.875, -28.25, -17.0, -22.75, -15.5, -22.5, -21.5, -13.375, -18.5, -18.125, -20.5, -17.75, -20.25, -20.0, -24.625, -26.5, -19.0, -27.0, -24.5, -28.5, -26.625, -20.375, -19.125, -31.875, -18.5, -23.25, -27.0, -25.25, -23.625, -20.75, -29.875, -19.875, -29.125, -27.875, -22.875, -17.875, -18.5, -18.875, -14.4375, -25.5, -19.0, -19.5, -21.625, -28.875, -26.0, -25.375, -24.5, -10.625, -23.0, -21.875, -14.5625, -29.125, -24.5, -19.625, -21.125, -14.75, -30.125, -19.25, -22.25, -24.5, -32.5, -19.125, -28.5, -26.125, -18.125, -21.625, -23.5, -25.375, -17.0, -18.875, -18.75, -26.375, -19.875, -19.625, -25.375, -14.6875, -24.0, -22.125, -15.4

In [9]:
import numpy as np


In [10]:
np.exp(-0.016434656456112)

0.9836996557122539